In [1]:
import sklearn
import numpy as np
import pandas as pd
from dev import DEVALGO 
from smote import SMOTEBoost
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score

In [2]:
MINORITY = 1
MAJORITY = 0
CLASS_NAME = "defects"

In [3]:
def pprint_dict(d):
    for key, value in d.items():
        print(key + ": " + "{0:.2f}".format(value/100.0))

In [4]:
df = pd.read_csv("data/PC1/pc1.csv").sample(frac=1, random_state=0).reset_index(drop=True)
df.head()

,loc,v(g),ev(g),iv(G),N,V,L,D,I,E,...,lOCode,lOComment,locCodeAndComment,lOBlank,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,27.0,19.0,17.0,3.0,238.0,1291.45,0.02,46.10,28.02,59529.74,...,27,0,0,3,22.0,21.0,150.0,88.0,37.0,False
1,11.0,4.0,3.0,2.0,62.0,312.75,0.08,12.71,24.61,3973.80,...,11,0,0,1,16.0,17.0,35.0,27.0,7.0,False
2,17.0,7.0,6.0,6.0,110.0,568.69,0.08,12.75,44.60,7250.82,...,17,0,0,1,12.0,24.0,59.0,51.0,13.0,False
3,12.0,3.0,1.0,3.0,22.0,85.95,0.22,4.50,19.10,386.78,...,9,2,3,5,9.0,6.0,16.0,6.0,4.0,False
4,13.0,4.0,1.0,4.0,60.0,291.48,0.09,10.56,27.60,3078.75,...,12,0,1,2,13.0,16.0,34.0,26.0,7.0,False


In [5]:
# Outcomes distribution
df[CLASS_NAME].value_counts()

False    1032
True       77
Name: defects, dtype: int64

In [6]:
# Uncomment this when testing on vowel
df[CLASS_NAME].replace({False:0, True:1}, inplace=True)
df.head()

,loc,v(g),ev(g),iv(G),N,V,L,D,I,E,...,lOCode,lOComment,locCodeAndComment,lOBlank,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,27.0,19.0,17.0,3.0,238.0,1291.45,0.02,46.10,28.02,59529.74,...,27,0,0,3,22.0,21.0,150.0,88.0,37.0,0
1,11.0,4.0,3.0,2.0,62.0,312.75,0.08,12.71,24.61,3973.80,...,11,0,0,1,16.0,17.0,35.0,27.0,7.0,0
2,17.0,7.0,6.0,6.0,110.0,568.69,0.08,12.75,44.60,7250.82,...,17,0,0,1,12.0,24.0,59.0,51.0,13.0,0
3,12.0,3.0,1.0,3.0,22.0,85.95,0.22,4.50,19.10,386.78,...,9,2,3,5,9.0,6.0,16.0,6.0,4.0,0
4,13.0,4.0,1.0,4.0,60.0,291.48,0.09,10.56,27.60,3078.75,...,12,0,1,2,13.0,16.0,34.0,26.0,7.0,0


In [7]:
# Calculate imbalanced ratio
frq = df[CLASS_NAME].value_counts()
ratio = max(frq)/min(frq)
"{0:.1f}".format(ratio)

'13.4'

# Use the following for testing Decision tree

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [8]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train, y_train)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [9]:
pprint_dict(res)

precision: 0.25
recall: 0.27
f1-score: 0.26
specificity: 0.94
sensitivity: 0.27
overall accuracy: 0.90
auc: 0.61
g_mean: 0.50


# Use the following for testing SMOTE

- run 100 iterations
- 50/50 training/testing
- Decision tree
- N = 200

In [10]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    sm = SMOTE(random_state=0)
    sm.fit(X_train, y_train)
    X_train_r, y_train_r = sm.fit_resample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (554, 21)
Outcome distribution of X_train before oversampling: [514  40]
Shape of X_train after oversampling: (1028, 21)
Outcome distribution of X_train after oversampling: [514 514]


In [11]:
pprint_dict(res)

precision: 0.22
recall: 0.38
f1-score: 0.27
specificity: 0.90
sensitivity: 0.38
overall accuracy: 0.87
auc: 0.64
g_mean: 0.58


# Use the following for testing ADASYN

- run 100 iterations
- 50/50 training/testing
- Decision tree
- A fully balanced dataset after synthesizing
- Dth = 0.75 (Dth is a preset threshold for the maximum tolerated degree of class imbalance ratio)

In [12]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    if i == 0:
        print("Shape of X_train before oversampling: " + str(X_train.shape))
        print("Outcome distribution of X_train before oversampling: " + str(np.bincount(y_train)))
    # Oversample training data
    ada = ADASYN(random_state=0)
    ada.fit(X_train, y_train)
    X_train_r, y_train_r = ada.fit_resample(X_train, y_train)
    if i == 0:
        print("Shape of X_train after oversampling: " + str(X_train_r.shape))
        print("Outcome distribution of X_train after oversampling: " + str(np.bincount(y_train_r)))
    # Build classifier on resampled data
    clf_tree = DecisionTreeClassifier(random_state=0)
    clf_tree.fit(X_train_r, y_train_r)
    y_pred = clf_tree.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

Shape of X_train before oversampling: (554, 21)
Outcome distribution of X_train before oversampling: [514  40]
Shape of X_train after oversampling: (1039, 21)
Outcome distribution of X_train after oversampling: [514 525]


In [13]:
pprint_dict(res)

precision: 0.26
recall: 0.51
f1-score: 0.35
specificity: 0.90
sensitivity: 0.51
overall accuracy: 0.87
auc: 0.71
g_mean: 0.68


# Use the following for testing SMOTEBoost

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [14]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    clf1 = SMOTEBoost(random_state=0)
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [15]:
pprint_dict(res)

precision: 0.16
recall: 0.41
f1-score: 0.23
specificity: 0.85
sensitivity: 0.41
overall accuracy: 0.82
auc: 0.63
g_mean: 0.59


# Use the following for testing Dev_algo

- run 100 iterations
- 50/50 training/testing
- Decision tree

In [16]:
res = dict()
X, y= df.iloc[:,:-1].values, df[CLASS_NAME].values
for i in range(100):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.5, random_state=0)
    unique, counts = np.unique(y_train, return_counts=True)
    frequency = dict(zip(unique, counts))
    clf1 = DEVALGO(random_state=0, n_samples=frequency[MAJORITY])
    clf1.fit(X_train, y_train)
    y_pred = clf1.predict(X_test)
    tmp_res = classification_report(y_test, y_pred, output_dict=True)
    res["precision"] = res.get("precision", 0) + tmp_res["1"]["precision"]
    res["recall"] = res.get("recall", 0) + tmp_res["1"]["recall"]
    res["f1-score"] = res.get("f1-score", 0) + tmp_res["1"]["f1-score"]
    res["specificity"] = res.get("specificity", 0) + tmp_res[str(MAJORITY)]["recall"]
    res["sensitivity"] = res.get("sensitivity", 0) + tmp_res[str(MINORITY)]["recall"]
    res["overall accuracy"] = res.get("overall accuracy", 0) + accuracy_score(y_test, y_pred,)
    res["auc"] = res.get("auc", 0) + roc_auc_score(y_test, y_pred)
    res["g_mean"] = res.get("g_mean", 0) + geometric_mean_score(y_test, y_pred)

In [17]:
 pprint_dict(res)

precision: 0.34
recall: 0.30
f1-score: 0.32
specificity: 0.96
sensitivity: 0.30
overall accuracy: 0.91
auc: 0.63
g_mean: 0.53
